### Similarity graph construction experiment on SIFT100K

In [ ]:
import os, sys
import os.path as osp
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=0
sys.path.append('..')

from torch.utils.tensorboard import SummaryWriter
import numpy as np
import torch
import time
import lib

print("Numpy: {}, Torch: {}".format(np.__version__, torch.__version__))

### Download vertices, graph edges and ground truth neighbors

In [ ]:
DATA_DIR = './data/SIFT100K'

if not osp.exists(DATA_DIR):
    assert not DATA_DIR.endswith(os.sep), 'please do not put "/" at the end of DATA_DIR'
    !mkdir -p {DATA_DIR}
    !wget https://www.dropbox.com/sh/vw6ojdvldg3ph1k/AABp2J2_r7FajjDvnqTgGKvSa?dl=1 -O {DATA_DIR}/sift_100k.zip
    !cd {DATA_DIR} && unzip sift_100k.zip && rm sift_100k.zip

### Configuration

NOTE: this config requires ~10.5GB GPU memory

In [ ]:
################
# Graph params #
################

graph_type = 'nsw'        # 'nsw' or 'nsg'
M = 12                    # degree parameter for NSW (Max degree is 2*M)
ef = 5                    # search algorithm parameter, sets the operating point
R = 16                    # degree parameter for NSG (corresponds to max degree)
k = 1                     # Number of answers per query. Need for Recall@k

assert k <= ef

nn = 200                  # Number of NN in initial KNNG that is used for NSG construction 
efC = 300                 # efConstruction used for NSW graph
ngt = 100                 # Number of ground truth answers per query
val_queries_size = 20000  # Number of queries for validation

#################
# Reward params #
#################

max_dcs = 800             # reward hyperparameter

################
# Agent params #
################

hidden_size = 2048        # number of hidden units

####################
# Algorithm params #
####################

samples_in_batch = 90000  # Reduce for larger hidden_size to fit in GPU memory 
Fvp_speedup = 5           # fraction of samples for Fisher vector product estimation 
                          # Reflects on the iteration time (<10 is okay)
Fvp_type = 'fim'          # Fisher vector product implementation: ['forward', 'fim']
entropy_reg = 0.01        # coefficient in front of the entropy regularizer term 
batch_size = 50000        # number of sessions per batch

n_jobs = 8                # Number of threads for C++ sampling
max_steps = 1000          # Max number of training iterations

In [ ]:
import lib

graph_params = { 
    'vertices_path': osp.join(DATA_DIR, 'sift_base.fvecs'),

    'train_queries_path': osp.join(DATA_DIR, 'sift_learn.fvecs'),
    'test_queries_path': osp.join(DATA_DIR, 'sift_query.fvecs'),
    
    'train_gt_path': osp.join(DATA_DIR, 'train_gt.ivecs'),
    'test_gt_path': osp.join(DATA_DIR, 'test_gt.ivecs'),
#     ^-- comment these 2 lines to re-compute ground truth ids (if you don't have pre-computed ground truths)
    
    'val_queries_size': val_queries_size,
    'ground_truth_n_neighbors': ngt,  # for each query, finds this many nearest neighbors via brute force
    'graph_type': graph_type
}

if graph_type == 'nsg':
    graph_params['edges_path'] = osp.join(DATA_DIR, 'sift_R{R}_{nn}nn.nsg'.format(R=R, nn=nn))
elif graph_type == 'nsw':
    graph_params['edges_path'] = osp.join(DATA_DIR, 'sift_nsw_M{M}_efC{efC}.ivecs'.format(M=M, efC=efC))
    graph_params['initial_vertex_id'] = 0 # by default, starts search from this vertex
else:
    raise ValueError("Wrong graph type: ['nsg', 'nsw']")
    
graph = lib.Graph(**graph_params)

In [ ]:
if graph_type == 'nsw':
    exp_name = '{data_name}_{graph_type}_k{k}_M{M}_ef{ef}_max-dcs{max_dcs}_hid-size{hidden_size}_entropy{entropy_reg}'.format(
        data_name=osp.split(DATA_DIR)[-1], k=k, M=M, ef=ef, hidden_size=hidden_size, 
        max_dcs=max_dcs, entropy_reg=entropy_reg, graph_type=graph_type
    )
elif graph_type == 'nsg':
    exp_name = '{data_name}_{graph_type}_k{k}_R{R}_ef{ef}_max-dcs{max_dcs}_hid-size{hidden_size}_entropy{entropy_reg}'.format(
        data_name=osp.split(DATA_DIR)[-1], k=k, R=R, ef=ef, hidden_size=hidden_size, 
        max_dcs=max_dcs, entropy_reg=entropy_reg, graph_type=graph_type
    )
else:
    raise ValueError("Wrong graph type: ['nsg', 'nsw']")
    
print('exp name:', exp_name)
#!rm {'./runs/' + exp_name} -rf # KEEP COMMENTED!
assert not os.path.exists('./runs/' + exp_name)

### HNSW, Agent, Reward, Baseline and Trainer

In [ ]:
hnsw = lib.ParallelHNSW(graph, ef=ef, k=k, n_jobs=n_jobs)
agent = lib.SimpleNeuralAgent(graph.vertices.shape[1], hidden_size=hidden_size)
reward = lib.MaxDCSReward(k=k, max_dcs=max_dcs)
baseline = lib.SessionBaseline(graph.train_queries.size(0))
trainer = lib.EfficientTRPO(agent, hnsw, reward, baseline,
                            samples_in_batch=samples_in_batch,
                            Fvp_type=Fvp_type,
                            Fvp_speedup=Fvp_speedup, entropy_reg=entropy_reg,
                            writer=SummaryWriter('./runs/' + exp_name))

### Training loop

In [ ]:
from pandas import DataFrame
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
moving_average = lambda x, **kw: DataFrame({'x':np.asarray(x)}).x.ewm(**kw).mean().values
reward_history = []

# generate batches of [queries, ground truth, train_query_ids (for baseline)]
train_query_ids = torch.arange(graph.train_queries.size(0))
train_batcher = lib.utils.iterate_minibatches(graph.train_queries, graph.train_gt, train_query_ids, 
                                              batch_size=batch_size)

# generate batches of [queries, ground truth]           
val_iterator = lib.utils.iterate_minibatches(graph.val_queries, graph.val_gt, 
                                             batch_size=graph.val_queries.size(0))

dev_iterator = lib.utils.iterate_minibatches(graph.test_queries, graph.test_gt, 
                                             batch_size=graph.test_queries.size(0))

In [ ]:
for batch_queries, batch_gt, batch_query_ids in train_batcher:
    start = time.time()
    torch.cuda.empty_cache()
    mean_reward = trainer.train_step(batch_queries, batch_gt, query_index=batch_query_ids)
    reward_history.append(mean_reward)
        
    if trainer.step % 10 == 0:
        trainer.evaluate(*next(val_iterator), prefix='val')
        
    if trainer.step % 50 == 0:
        trainer.evaluate(*next(dev_iterator))
        print(end="Saving...")
        torch.save(agent, "runs/{}/agent.{}.pth".format(exp_name, trainer.step))
        torch.save(baseline, "runs/{}/baseline.{}.pth".format(exp_name, trainer.step))
        print('Done!')
    
    if trainer.step % 1 == 0:
        clear_output(True)
        plt.title('train reward over time')
        plt.plot(moving_average(reward_history, span=50))
        plt.scatter(range(len(reward_history)), reward_history, alpha=0.1)
        plt.grid()
        plt.show()
        print("step=%i, mean_reward=%.3f, time=%.3f" % 
              (trainer.step, np.mean(reward_history[-100:]), time.time()-start))
    
    if trainer.step >= max_steps: break

#protip: run tensorboard in ./runs to get all metrics.

In [ ]:
best_val_step = None

if best_val_step is not None:
    agent = torch.load("runs/{}/agent.{}.pth".format(exp_name, best_val_step))
    trainer.step = best_val_step

In [ ]:
from collections import defaultdict

agent.cuda()
state = agent.prepare_state(graph, device='cuda')

new_edges = defaultdict(list)
for vector_id in graph.edges.keys():
    list_edges = list(graph.edges[vector_id])
    d = len(list_edges)
    with torch.no_grad():
        edges_logp = agent.get_edge_logp([vector_id]*d, list_edges, state=state, device='cuda').cpu()
        edges_mask = edges_logp.argmax(-1).numpy() == 1
    new_edges[vector_id] = [edge for i, edge in enumerate(list_edges) if edges_mask[i] == 1]

#Save constructed graph
lib.write_edges("runs/{}/graph.{}.ivecs".format(exp_name, trainer.step), new_edges)

### Evaluate constructed graph

In [ ]:
import sys

for heap_size in [1, 2, 3, 5, 10, 20, 30]:
    algo_hnsw = lib.BaseAlgorithm(
        agent=agent,
        hnsw=lib.ParallelHNSW(graph, ef=heap_size, k=k, n_jobs=n_jobs),
        reward=lambda actions, **kw: [0] * len(actions),
        writer=trainer.writer, device='cuda',
    )
    algo_hnsw.step = trainer.step  # for tensorboard

    metrics = algo_hnsw.get_session_batch(graph.test_queries, graph.test_gt, greedy=True,
                             summarize=True, write_logs=False, prefix='dev',)['summary']
    sys.stderr.flush()
    print("Ef %i | Recall@%d %f | Distances: %f" % 
          (heap_size, k, metrics['dev/recall@%d' % k], metrics['dev/distance_computations']),
          flush=True,
         )